In [40]:
import pandas as pd
import numpy as np

from scipy.sparse import hstack

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge

In [3]:
'''
   1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла 
   salary-train.csv (либо его заархивированную версию salary-train.zip).
'''

df = pd.read_csv('salary-train.csv')

In [67]:
#2. Проведите предобработку: 

# Приведите тексты к нижнему регистру (text.lower())

df['LocationNormalized'] = df['LocationNormalized'].map(lambda i : i.lower())
df['FullDescription'] = df['FullDescription'].map(lambda i : i.lower())

'''
    Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение
    текста на слова. Для такой замены в строке text подходит следующий вызов: 
    re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у 
    DataFrame, чтобы сразу преобразовать все тексты:
            train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', 
            regex = True)
'''

df['FullDescription'] = df['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
df['LocationNormalized'] = df['LocationNormalized'].replace('[^a-zA-Z0-9]', ' ', regex = True)

# Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.

df['LocationNormalized'].fillna('nan', inplace=True)
df['ContractTime'].fillna('nan', inplace=True)


'''
    Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте 
    только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
'''

vec = TfidfVectorizer(min_df = 5)
X_train_text = vec.fit_transform(df['FullDescription'])

# Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

enc = DictVectorizer()
X_train_cat = enc.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))

'''
    Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и
    категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией
    scipy.sparse.hstack.
'''

X_train = hstack([X_train_text, X_train_cat])

In [47]:
'''
    3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана 
    в столбце SalaryNormalized.
'''

y_train = df['SalaryNormalized']
model = Ridge(alpha=1.0, random_state=241)
model.fit(X_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [68]:
'''
    4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов
    являются ответом на задание. Укажите их через пробел.
'''

test = pd.read_csv('salary-test-mini.csv')

test['FullDescription'] = test['FullDescription'].map(lambda i : i.lower())
test['FullDescription'] = test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

X_test_text = vec.transform(test['FullDescription'])
X_test_cat = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([X_test_text, X_test_cat])

y_test = model.predict(X_test)

In [70]:
print('{:0.2f} {:0.2f}'.format(y_test[0], y_test[1]))

55850.59 37182.51
